In [ ]:
import numpy as np
import re

In [ ]:
rawdata = """\
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
""".split("\n\n")
rawflows, rawinput = rawdata
rawflows, rawinput

In [ ]:
with open("input.txt") as f:
    rawflows, rawinput = f.read().split("\n\n")

In [ ]:
rawflows.split("\n")

In [ ]:
def parse_flow(rawflows):
    flows = dict()
    for line in rawflows.split("\n"):
        key, flowraw = line.rstrip("}").split("{")
        #print(key, flowraw)
        steps = []
        pattern = r'(\w+|[<>])'
        for step in flowraw.split(","):
            if ":" in step:
                condraw, name = step.split(":")
                cond = re.findall(pattern, condraw)
                cond[2] = int(cond[2])
                steps.append([name, cond])
            else:
                steps.append([step])
        flows[key] = steps
    return flows

In [ ]:
flows = parse_flow(rawflows)
flows['in']

In [ ]:
def parse_input(inp):
    V = inp.strip("{}").split(",")
    return [ int(item.split("=")[-1]) for item in V ]
parts = [ parse_input(inp) for inp in rawinput[:-1].split("\n") ]
parts[-3:]

## question 1

In [ ]:
def get_new_flow(part, flow):
    I = {'x':0, 'm':1, 'a':2, 's':3}
    for key, condition in flow[:-1]:
        cat, op, v = condition # category, operation, value
        vp = part[I[cat]]
        #print(cat, op, v, vp)
        if (op=="<" and vp<v) or (op==">" and vp>v):
            new_flow = key
            return new_flow
    else:
        return flow[-1][0]

def get_state(part):
    #print(part)
    flow = flows['in']
    maxit, nit = 10, 0
    while True:
        new_flow_key = get_new_flow(part, flow)
        #print(part, new_flow_key)
        if new_flow_key == "A":
            return "A"
        elif new_flow_key == "R":
            return "R"
        else:
            flow = flows[new_flow_key]
        
        nit += 1
        if nit >= maxit:
            print("reached maxit")
            break
        print(nit, end=",")

In [ ]:
total = 0
for part in parts:
    if get_state(part) == "A":
        total += sum(part)
total

## question 2

In [ ]:
class Range(object):
    def __init__(self, start=1, end=4000):
        self.start = start
        self.end = end
    def split(self, op, cv): #operator condition_value
        # return in order of succes / no success
        # this has to split more carefully. also return [None, Range] or [Range, None] to indicate what the next step will be.
        if op=="<":
            if self.start>=cv:
                return [None, self]
            elif self.end<cv:
                return [self, None]
            else:
                return Range(start=self.start, end=cv-1), Range(start=cv, end=self.end)
        else:
            assert op==">"
            if self.start>cv:
                return [self, None]
            if self.end<=cv:
                return [None, self]
            else:
                return Range(start=cv+1, end=self.end), Range(start=self.start, end=cv)
    def __repr__(self):
        return f"<Range:{self.start}-{self.end}>"
    def __len__(self):
        return self.end - self.start + 1

In [ ]:
initial = [Range(), Range(), Range(), Range()]
initial

In [ ]:
accepted_ranges = []
I = {'x':0, 'm':1, 'a':2, 's':3}


def handle_ranges(ranges, flow):
    print(ranges, flow)
    
    for key, condition in flow[:-1]:
        cat, op, v = condition # category, operation, value
        ranges_success, ranges_fail = ranges[I[cat]].split(op, v)
        print("in for loop", cat, op, v, ranges_success, ranges_fail)
        if ranges_success: # get the new flowkey
            new_ranges = ranges.copy()
            new_ranges[I[cat]] = ranges_success
            if key == "A":
                accepted_ranges.append(new_ranges)
                return
            elif key== "R":
                return
            else:
                handle_ranges(new_ranges, flows[key])
        if ranges_fail: # the ranges with the fail part goes to next iteration
            ranges[I[cat]] = ranges_fail
    else: # last flow
        print("last flow with ranges", ranges)
        new_flow = flow[-1][0]
        if new_flow == "A":
            accepted_ranges.append(ranges)
            return
        elif new_flow == "R":
            return
        else:
            return handle_ranges(ranges, flows[new_flow])

In [ ]:
handle_ranges(initial, flows['in'])

In [ ]:
flows['in'], flows['px'], flows['qkq']

In [ ]:
accepted_ranges

In [ ]:
ntotal = 0
for accepted_range in accepted_ranges:
    lens = list(map(len, accepted_range))
    print(lens, np.prod(lens))
    ntotal += np.prod(lens)
ntotal

In [ ]:
# correct answer should be 
# 167409079868000
# 227579675642736
# 212938445310001
# 145370496664781